In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import simlib
from importlib import reload


In [ ]:
reload(simlib)
        
# SIMULATION CONTROLS ################
# LIFTOFF
STAGE_1_PROPELLANT_MASS = 120e3
ROTATION_ANGLE = -75

# HOHMANN TRANSFER
STAGE_2_TOTAL_MASS = 30e3
STAGE_2_FIRST_BURN_MASS = 0
STAGE_2_SECOND_BURN_MASS = 0
#######################################

MAX_STEPS = 30000
earth = simlib.Earth

if STAGE_2_SECOND_BURN_MASS == 0: MAX_STEPS = 15000
if STAGE_2_FIRST_BURN_MASS == 0: MAX_STEPS = 5000



h1 = 100
p1 = (0, earth.radius + h1) # (x0 (m), y0 (degrees))
theta1 = 0
v1 = np.array((np.cos(np.deg2rad(theta1 + 90)), np.sin(np.deg2rad(theta1 + 90)))) * 0.01 #

rocket = simlib.SpaceCraft(1e3 + STAGE_2_TOTAL_MASS - STAGE_2_FIRST_BURN_MASS - STAGE_2_SECOND_BURN_MASS, 
                           p1, v1, (0,0), 0, gridsize_min=10000, max_dt=10,
                           engines=((simlib.ariane_booster, None),
                                    (simlib.ariane_booster, None),
                                    (simlib.ariane_stage1, STAGE_1_PROPELLANT_MASS),
                                    (simlib.ariane_stage2, STAGE_2_FIRST_BURN_MASS),
                                    (simlib.ariane_stage2_second_burn, STAGE_2_SECOND_BURN_MASS),))

assert STAGE_2_FIRST_BURN_MASS + STAGE_2_SECOND_BURN_MASS <= STAGE_2_TOTAL_MASS, 'first burn mass + second burn mass higher than total mass'
print(f'TOTAL ROCKET MASS: {rocket.get_mass()/1e3} T')

def start_main_stage_and_boosters():
    rocket.start_thruster()
    rocket.start_thruster()
    rocket.start_thruster()
    

start_main_stage_and_boosters = simlib.Action(start_main_stage_and_boosters)
start_stage2_first_burn = simlib.Action(rocket.start_thruster)
start_stage2_second_burn = simlib.Action(rocket.start_thruster)
rotate = simlib.Action(rocket.rotate)
rotate2 = simlib.Action(rocket.rotate)

all_distance = list()
step_nb = 0
while True:
    step_nb += 1
    
    P1, R1, g1, h1 = earth.get_PRgh(rocket.p, rocket.v)
    if h1 < 110:
        g1 = 0
    
    if 1 < rocket.t:
        start_main_stage_and_boosters()

    if h1 >= 500e3:
        rotate(ROTATION_ANGLE)

    if rocket.t > 3000 and STAGE_2_FIRST_BURN_MASS > 0:
        start_stage2_first_burn()

    if h1 >= 35789e3 and STAGE_2_SECOND_BURN_MASS > 0:
        start_stage2_second_burn()

    dt = rocket.update(P1, R1, g1, R1+h1)

    
    if step_nb > MAX_STEPS:
        print('==== TOO MUCH STEPS ====')
        break

    if h1 < 0:
        print('==== rocket CRASHED !! ====')
        break

rocket.plot_trajectory(earth, altitude_target=(500e3, 35789e3))

plt.figure()
rocket.plot_velocity()
rocket.plot_acceleration()
rocket.plot_mass()

print(f"""TOTAL FUEL USED:
    BOOSTER: 2 x 240 T
    STAGE1: {STAGE_1_PROPELLANT_MASS/1e3} T
    STAGE2
        TOTAL: {STAGE_2_TOTAL_MASS/1e3} T
        FIRST BURN: {STAGE_2_FIRST_BURN_MASS/1e3} T
        SECOND BURN: {STAGE_2_SECOND_BURN_MASS} T
        UNUSED: {STAGE_2_TOTAL_MASS - STAGE_2_FIRST_BURN_MASS - STAGE_2_SECOND_BURN_MASS}""")